<a href="https://colab.research.google.com/github/MarrtinJ/fantasy-bball-opt/blob/main/bruteForceLineups2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing/Importing Necessary Libraries

In [41]:
import numpy as np
import pandas as pd

In [42]:
# stop the pandas indexing/splicing warning from appearing
import warnings
warnings.filterwarnings('ignore')

In [43]:
df = pd.read_csv('dataset2.csv')
df = df[['Date', 'Name', 'Team', 'Position', 'Salary', 'FPTS']]
df = df[df.FPTS >= 0] # drop all players who scored less than 0 points
df.shape

(43291, 6)

In [44]:
# all the unique dates in the dataset
dates = df['Date'].unique()

# DFS Setup / Helper Functions

In [45]:
def buildPositionDFs(df1):
  # incr determines if the players are each position are sorted by increasing salary
  incr = [True, False]

  PGs = df1[df1['Position'] == 'PG']
  PGs.sort_values(by=['Salary', 'FPTS'], ascending=incr, inplace=True)
  PGs.reset_index(inplace=True, drop=True)


  SGs = df1[df1['Position'] == 'SG']
  SGs.sort_values(by=['Salary', 'FPTS'], ascending=incr, inplace=True)
  SGs.reset_index(inplace=True, drop=True)

  SFs = df1[df1['Position'] == 'SF']
  SFs.sort_values(by=['Salary', 'FPTS'], ascending=incr, inplace=True)
  SFs.reset_index(inplace=True, drop=True)

  PFs = df1[df1['Position'] == 'PF']
  PFs.sort_values(by=['Salary', 'FPTS'], ascending=incr, inplace=True)
  PFs.reset_index(inplace=True, drop=True)

  Cs = df1[df1['Position'] == 'C']
  Cs.sort_values(by=['Salary', 'FPTS'], ascending=incr, inplace=True)
  Cs.reset_index(inplace=True, drop=True)

  Gs = df1[df1['Position'] == 'G']
  Gs.sort_values(by=['Salary', 'FPTS'], ascending=incr, inplace=True)
  Gs.reset_index(inplace=True, drop=True)

  Fs = df1[df1['Position'] == 'F']
  Fs.sort_values(by=['Salary', 'FPTS'], ascending=incr, inplace=True)
  Fs.reset_index(inplace=True, drop=True)


  Utils = df1[df1['Position'] == 'UTIL']
  Utils.sort_values(by=['Salary', 'FPTS'], ascending=incr, inplace=True)
  Utils.reset_index(inplace=True, drop=True)

  return [PGs, SGs, SFs, PFs, Cs, Gs, Fs, Utils]

In [54]:
# returns a list of player names that are not pareto-dominated
def paretoFilter(posDF, singlePosNames):
  discard = []
  # print(singlePosNames[0])
  if not singlePosNames: # no players listed under solely one position
    return discard
  else:
    curBest = posDF[posDF['Name'] == singlePosNames[0]].iloc[0] # the first best player
    # print(curBest)
    for index, row in posDF.iterrows():
      # produces more fantasy points and is eligible to be the 'best'
      if row['FPTS'] > curBest['FPTS'] and row['Name'] in singlePosNames:
        curBest = row
        # print('Updating current best')
        # print(curBest)
      else:
        # if the current player costs more than the current best and produces less fantasy points
        if row['FPTS'] <= curBest['FPTS']:
            # print(row['Name'], row['Salary'], row['FPTS'])
            # print('Dropping {}'.format(row['Name']))
            # print('Cur: {}, {} Best:{} {}'.format(row['Salary'], row['FPTS'], curBest['Salary'], curBest['FPTS']))
            discard.append(row['Name'])
    return discard

In [47]:
def getSingleScore(player):
  return currentPlayers[currentPlayers['Name'] == player].iloc[0].FPTS

def getSingleSalary(player):
  return currentPlayers[currentPlayers['Name'] == player].iloc[0].Salary

In [48]:
# getScore takes in a dict of players, returns the fantasy points scored by players
def getScore(players):
  total_score = 0
  for player in players.values():
    player_score = getSingleScore(player)
    # print(player, player_score)
    total_score += player_score
  return total_score

**bold text**# Inverted DFS

In [60]:
MAX_POSITION_INDEX = 7

# invertedDFS(players, 0, 0, 50000)
def invertedDFS(players, pos_index, player_index, budget):
  # print(pos_index, len(players[pos_index]))
  player = players[pos_index][player_index]

  salary = getSingleSalary(player)
  newBudget = budget - salary

  # optimization 1: instead of comparing with 0, compare with sum(min_budget of each remaining position)
  #   can be computed ahead of time for each level
  if pos_index < MAX_POSITION_INDEX and newBudget < minSalaries[pos_index+1]:
    # print('out of budget at position {}'.format(pos_index))
    return dict(), np.NINF

  # optimization 3: two separate cases: if I have enough budget to pick the max salary from every remaining level, no need for recursion, just pick best score from every lower level
  # otherwise, do the recursive search in the next block
  if pos_index < MAX_POSITION_INDEX and newBudget > maxSalaries[pos_index+1]:
    # print('can afford best players at position {}'.format(pos_index))

    assignment = dict()
    assignment[pos_index] = player
    for i in range(pos_index+1, 8):
      for index, row in l3[i].iterrows():
        if row['Name'] not in assignment.values():
          assignment[i] = row['Name']
          break
    # print(assignment, getScore(assignment))
    return assignment, getScore(assignment)

  if pos_index == MAX_POSITION_INDEX:
    assignment = {pos_index: player}
    # print(pos_index, player_index, player)
    return assignment, getScore(assignment)
  else:
    max_score = 0
    best_lineup = dict()
    for j in range(len(players[pos_index + 1])):
    # for j in range(8):
      a, s = invertedDFS(players, pos_index+1, j, newBudget) 
      # optimization 2: if a returns None (a is empty), break out of the current loop
      if not a:
        # print('next level returned none')
        break
      if s > max_score and a and player not in a.values(): #duplicate check
        max_score = s
        best_lineup = a
        # print(max_score, best_lineup)
    if best_lineup:
      best_lineup[pos_index]=player
      # if pos_index==MAX_POSITION_INDEX:
      #   print(best_lineup, max_score+getSingleScore(player))
      return best_lineup, max_score + getSingleScore(player)
    else:
      return dict(), np.NINF

# Main Setup/Function Call

In [50]:
pos_dict = {0: 'PG', 1:'SG', 2:'SF', 3:'PF', 4:'C', 5:'G', 6:'F', 7:'UTIL'}

In [55]:
numLineups = 0

In [ ]:
players_per_day = []
data = []

# for x in range(len(dates)):
for x in range(1,2):
  date=dates[x]
  print(date)
  day = df[df['Date']==date]

  # print(day.shape[0])
  # players_per_day.append(day.shape[0])

  # if (day.shape[0] > 121):
  #   continue
  

  Utils = day.copy()
  Utils['Position'] = 'UTIL'
  
  onePosition = day[ day['Position'].str.contains('/')==False ]
  
  onePosition.sort_values(by=['Salary', 'FPTS'], ascending=[True, False], inplace=True)
  onePositionPG = onePosition[ onePosition['Position'] == 'PG' ]
  onePositionSG = onePosition[ onePosition['Position'] == 'SG' ]
  onePositionSF = onePosition[ onePosition['Position'] == 'SF' ]
  onePositionPF = onePosition[ onePosition['Position'] == 'PF' ]
  onePositionC = onePosition[ onePosition['Position'] == 'C' ]
  onePositionG = pd.concat([onePositionPG, onePositionSG], ignore_index=True)
  onePositionF = pd.concat([onePositionSF, onePositionPF], ignore_index=True)
  onePositionUtil = onePosition.copy()

  onePositionPG.drop_duplicates(subset='Salary', keep='first', inplace=True)
  onePositionSG.drop_duplicates(subset='Salary', keep='first', inplace=True)
  onePositionSF.drop_duplicates(subset='Salary', keep='first', inplace=True)
  onePositionPF.drop_duplicates(subset='Salary', keep='first', inplace=True)
  onePositionC.drop_duplicates(subset='Salary', keep='first', inplace=True)
  onePositionG.drop_duplicates(subset='Salary', keep='first', inplace=True)
  onePositionF.drop_duplicates(subset='Salary', keep='first', inplace=True)
  onePositionUtil.drop_duplicates(subset='Salary', keep='first', inplace=True)

  onePosList = [onePositionPG, onePositionSG, onePositionSF, onePositionPF, onePositionC, onePositionG, onePositionF, onePositionUtil]


  multPositions = day[day['Position'].str.contains("/")]
  multPositions.reset_index(inplace=True, drop=True)

  pos1 = []
  pos2 = []
  for index, row in multPositions.iterrows():
    playerPos1, playerPos2 = row['Position'].split('/')
    copy1 = row.copy()
    copy1['Position'] = playerPos1
    copy2 = row.copy()
    copy2['Position'] = playerPos2
    # print(copy)
    pos1.append(copy1)
    pos2.append(copy2)
    # print(playerPos1, playerPos2)

  pos1 = pd.DataFrame(pos1)
  pos2 = pd.DataFrame(pos2)
  
  currentPlayers = pd.concat([onePosition, pos1, pos2], ignore_index=True)
  currentPlayers.sort_values(['Name', 'Team', 'Position'], na_position='first', inplace=True, ignore_index=True)

  Gs = currentPlayers[currentPlayers['Position'].isin(['PG', 'SG'])]
  Gs.drop_duplicates(subset='Name', inplace=True)
  Gs['Position'] = 'G'

  Fs = currentPlayers[currentPlayers['Position'].isin(['SF', 'PF'])]
  Fs.drop_duplicates(subset='Name', inplace=True)
  Fs['Position'] = 'F'

  currentPlayers = pd.concat([currentPlayers, Gs, Fs, Utils], ignore_index=True)
  currentPlayers.sort_values(['Name', 'Team', 'Position'], na_position='first', inplace=True, ignore_index=True)

  l1 = buildPositionDFs(currentPlayers)
  
  # find pareto front, drop players that will never be in best solution
  for y in range (8):
    onePosNames = onePosList[y].Name.to_list()
    # print(onePosNames)
    # print('before: {}'.format(l1[y].shape))
    test = paretoFilter(l1[y], onePosNames)
    l1[y] = l1[y][~l1[y]['Name'].isin(test)]
    # print(len(test))
    # print('{} after: {}'.format(pos_dict[y], l1[y].shape))

  # minSalaries holds the minimum budget required to keep searching at each position
  #   where the number at index i represents the sum of the least expensive players at position index >= i

  # maxSalaries holds the minimum budget required to afford the highest scoring players in the remaining positions
  #   where the number at index i represents the sum of the most expensive players at position index >= i

  minSalaries = []
  maxSalaries = []
  l2 = [] # holds the dataframes for each positon, sorted in increasing order by max salary
  position_order = []

  l3 = [] # holds the dataframes for each positon, sorted in decreasing order by fpts

  for i in range(8):
    # could also consider sorting by the mean salary of each position
    pos_max = l1[i].Salary.max()
    l2.append((pos_max, l1[i]))

  # sort the positions in descending order by max salary
  l2 = sorted(l2, key=lambda x: x[0], reverse=True)

  cheapest_sum = 0
  expensive_sum = 0
  for i in range(len(l2)):
    # keeping track of the sum of salaries for the most expensive players in each position
    pos_max = l2[i][0]
    expensive_sum += pos_max
    maxSalaries.insert(0, expensive_sum)

    # removing the max sal info, retaining just the dataframe
    l2[i] = l2[i][1]

    l3.append(l2[i].sort_values(by=['FPTS'], ascending=False))

    # keeping track of the sum of salaries for the least expensive players in each position
    pos_min = l2[i].Salary.min()
    cheapest_sum += pos_min
    minSalaries.insert(0, cheapest_sum)

    # keeping track of the sorted order of the positions
    pos = l2[i]['Position'].iloc[0]
    position_order.append(pos)

    # print(pos, pos_min, pos_max)

  # players[i][j] represents the ith player in the jth position
  # players[i] sorted in ascending order by salary

  players = []

  for i in range(len(l2)):
    players_list = l2[i]['Name'].to_list()
    # print()
    # print(position_order[i], players_list)
    players.append(players_list)

  for j in range(len(players)):
    print(position_order[j], len(players[j]))

  # continue 
  res = invertedDFS(players, 0, 0, 50000)

  soln = res[0]
  soln['Date'] = date
  soln['FPTS'] = res[1]
  print(soln)
  data.append(soln)
  print(f'done with {date}')

# Test

In [36]:
players_per_day = np.array(players_per_day)

In [38]:
players_per_day.mean()

121.60393258426966

In [ ]:
res

({}, -inf)

In [ ]:
  # soln_df = pd.DataFrame(data)
  # print(soln_df)
  # soln_df.to_csv(f'./data/soln/{date}.csv', line_terminator='\n', index=False)